In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# 讀取CSV數據
data = pd.read_csv("./data/2330.TW.csv")

# 選擇特徵和目標變量
features = data[["Open", "High", "Low", "Volume", "Adj Close"]]
target = data["Close"]

# 將數據縮放到0和1之間
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

# 拆分訓練集和測試集
train_x, test_x, train_y, test_y = train_test_split(scaled_features, target, test_size=0.2, random_state=42)

# 定義神經網絡模型
model = Sequential([
    Dense(64, input_dim=train_x.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

# model.compile(optimizer="sgd", loss="mean_absolute_error")
model.compile(optimizer="adam", loss="mean_squared_error")

# 使用EarlyStopping來防止過擬合
callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# 訓練模型
model.fit(
    train_x,
    train_y,
    validation_split=0.2,
    epochs=50,
    batch_size=50,
    callbacks=[callback]
)

# 預測並評估模型
preds = model.predict(test_x)
mse = np.mean((preds.flatten() - test_y)**2)
print(f"Mean Squared Error: {mse:.4f}")

# 如果需要將預測結果轉換回原始縮放，可以反轉縮放
# 但是這裏的目標是Close值，無需縮放，只需檢查預測結果即可


Epoch 1/50


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 381216.5625 - val_loss: 379013.9375
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 379599.3125 - val_loss: 378875.7188
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 385451.0625 - val_loss: 378732.6562
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 383746.1875 - val_loss: 378573.7500
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 385148.0312 - val_loss: 378404.6562
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 386034.3125 - val_loss: 378222.9062
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 379241.7188 - val_loss: 378026.0938
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 381548.5938 - val_loss: 377815.5938
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 377946.6875 - val_loss: 377579.2188
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 378019.3125 - val_loss: 377309.9375
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 379236.8750 - val_loss: 377007.6

In [27]:
# 計算準確率
# tolerance = 0.01# 設置5%的容忍範圍
correct_preds = np.abs(preds.flatten() - test_y) / test_y 
accuracy = np.mean(correct_preds) * 100
print(f"Accuracy: {accuracy:.2f} %")

#adam>5%
#sgd>10%
#mse>10%
#mae>0.4%

Accuracy: 89.28 %


In [28]:
#save
model.save("stock_prediction_model.h5")

In [29]:
from tensorflow.keras.models import load_model
#mse
new_model = load_model("stock_prediction_model.h5")
preds = new_model.predict(test_x)
mse = np.mean((preds.flatten() - test_y)**2)
print(f"MSE : {mse:.4f}")
#acc
correct_preds = np.abs(preds.flatten() - test_y) / test_y 
accuracy = np.mean(correct_preds) * 100
print(f"Acc : {accuracy:.2f} %")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
MSE : 281574.6627
Acc : 89.28 %


In [30]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense

new_train_x=train_x
new_train_y=train_y
# Step 1: Load the pre-trained model
pretrained_model = load_model('stock_prediction_model.h5')

# Step 2: Freeze some layers (optional)
for layer in pretrained_model.layers[:-2]: # Freeze all layers except the output layer
    layer.trainable = False

# Step 3: Add new layers
pretrained_model.add(Dense(16, activation='relu'))  # Add new dense layer
pretrained_model.add(Dense(1))  # Add output layer

# Step 4: Compile the model
pretrained_model.compile(optimizer="Adagrad", loss='mean_squared_error')

# Step 5: Train the model with new data
pretrained_model.fit(new_train_x, new_train_y,
          epochs=50,
          batch_size=30,
          validation_split=0.2)

#mse
preds = pretrained_model.predict(test_x)
mse = np.mean((preds.flatten() - test_y)**2)
print(f"MSE : {mse:.4f}")
#acc
correct_preds = np.abs(preds.flatten() - test_y) / test_y 
accuracy = np.mean(correct_preds) * 100
print(f"Acc : {accuracy:.2f} %")


Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 363840.0625 - val_loss: 366109.8125
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 361765.1875 - val_loss: 365329.2812
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 370295.0000 - val_loss: 364788.2188
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 367926.3438 - val_loss: 364296.4062
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 365880.8125 - val_loss: 363871.3750
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 366587.0938 - val_loss: 363487.6562
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 371764.5625 - val_loss: 363138.8125
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 358096.9375 - val_loss: 362773.2188
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 361684.0312 - val_loss: 362458.8125
Epoch 10/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 363248.5000 - val_loss: 362143.7812
Epoch 11/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 370904.0938 - val_los

In [31]:
#save
model.save("stock_prediction_model_v2.h5")